In [7]:
import datetime
import os.path

import time

# google api(календарик)
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# beautiful soup(парсить)
from bs4 import BeautifulSoup
import requests

# selenium т.к. оказалось, что страница ruz подгружает кучу инфы при помощи JS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# ИМЯ - меняем на своё
my_name = "Лунев Максим Александрович"

# Если это меняем, то удаляем token.json(меняем тогда, когда меняем scopes в дешборде).
SCOPES = ['https://www.googleapis.com/auth/calendar','https://www.googleapis.com/auth/calendar.events','https://www.googleapis.com/auth/calendar.settings.readonly','https://www.googleapis.com/auth/calendar.freebusy']

In [8]:
CREDS_FILE = "credentials.json"

In [9]:
def get_creds():
    creds = None
    # token.json генерится при входе и содержит права и креденты пользователя
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json','w') as token:
            token.write(creds.to_json())
    
    service = build('calendar', 'v3', credentials=creds)
    return service

In [10]:
service = get_creds()
print('Получаем список календарей')
calendars = service.calendarList().list().execute().get('items',[])

if not calendars:
    print('Календарей не найдено')
for calendar in calendars:
    summ = calendar['summary']
    id = calendar['id']
    primary = "Primary" if calendar.get('primary') else ""
    print("%s\t%s\t%s" % (summ, id, primary))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=377600499619-0fgbqe7vv32djqr8g33kau766cj2oe1j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A29148%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.events+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.settings.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.freebusy&state=B9yRQPYFIWqgBlG3cyCOnynnoervVf&access_type=offline
Получаем список календарей
liliputer517@gmail.com	liliputer517@gmail.com	Primary
Holidays in Russia	en.russian#holiday@group.v.calendar.google.com	
Birthdays	addressbook#contacts@group.v.calendar.google.com	


In [54]:
now = datetime.datetime.utcnow().isoformat() + 'Z'
events = service.events().list(calendarId='primary', timeMin=now, singleEvents=True).execute()
for event in events['items']:
    print(event['id'] + ' - ' + event['summary'])

099ph3ho74l27mtfd3ptndkko8 - test event


In [55]:
upd_event = service.events().get(calendarId='primary', eventId='099ph3ho74l27mtfd3ptndkko8').execute()
upd_event['summary'] = upd_event['summary']+'(upd)'
service.events().update(calendarId='primary', eventId='099ph3ho74l27mtfd3ptndkko8', body=upd_event).execute()

{'kind': 'calendar#event',
 'etag': '"3286463698388000"',
 'id': '099ph3ho74l27mtfd3ptndkko8',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=MDk5cGgzaG83NGwyN210ZmQzcHRuZGtrbzggbGlsaXB1dGVyNTE3QG0',
 'created': '2022-01-26T20:25:28.000Z',
 'updated': '2022-01-26T21:17:29.194Z',
 'summary': 'test event(upd)',
 'creator': {'email': 'liliputer517@gmail.com', 'self': True},
 'organizer': {'email': 'liliputer517@gmail.com', 'self': True},
 'start': {'dateTime': '2022-01-27T17:00:00+03:00',
  'timeZone': 'Europe/Moscow'},
 'end': {'dateTime': '2022-01-27T18:00:00+03:00', 'timeZone': 'Europe/Moscow'},
 'iCalUID': '099ph3ho74l27mtfd3ptndkko8@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True},
 'eventType': 'default'}

In [37]:
parse_url = 'https://ruz.hse.ru/ruz/main'
# посылаем GET запрос
ruz = requests.get(parse_url)
# если код 200 то всё работает
print(ruz.status_code)

200


In [44]:
# бахнем сюда selenium
driver = webdriver.Edge()
driver.get(parse_url)

# вбиваем группу и получаем расписание
time.sleep(3)
buttons = driver.find_elements(By.XPATH,"//*[contains(text(), 'Студент')]")
print(buttons)
buttons[0].click()
time.sleep(1)
group = driver.find_element(By.ID, 'autocomplete-student')
group.send_keys(my_name)
time.sleep(3)
group.send_keys(Keys.RETURN)
tmp = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.day')))
time.sleep(2)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body_code = body.get_attribute('innerHTML')

# заливаем уже полностью прогруженную страницу в переменную
# и парсим данные с помощью bf4

driver.quit()

[<selenium.webdriver.remote.webelement.WebElement (session="9025dcea0bc49421da622e75768ec857", element="9c6fcf54-8d28-444a-aa75-b301d86f0749")>, <selenium.webdriver.remote.webelement.WebElement (session="9025dcea0bc49421da622e75768ec857", element="0f20063a-6adb-46d9-84da-d260e93676a2")>]


In [156]:
ruz = BeautifulSoup(body_code, 'lxml')
days = ruz.find_all('div', class_='media day ng-star-inserted')
classes = []
# каждое занятие имеет класс day, поэтому в переменной day хранится какой-то определенный урок
class_i = 0
for day in days:
    #print(day)
    for class_ in day.select("span[class=ng-star-inserted]"):
        classes.append([class_.text])
    # получаем dates_ - все span занятия, а именно дата, день недели, время
    dates_ = day.select("div.d-lg-none.date.clearfix")
    for i in dates_:
        time = i.select("span")[2]
        date = i.select("span")[0]
        classes[class_i].append(time.text)
        classes[class_i].append(date.text)
    class_i+=1

In [157]:
# получили все данные для расписания, теперь их нужно привести в формат,
# читаемый для google calendar api
classes

[[' Введение в программирование на Java (рус) ',
  ' 13:00\xa0-\xa014:20 ',
  ' 24.01.2022 '],
 [' Дискретная математика (рус) ', ' 16:20\xa0-\xa017:40 ', ' 24.01.2022 '],
 [' Математический анализ (рус) ', ' 11:10\xa0-\xa012:30 ', ' 25.01.2022 '],
 [' Проектный семинар "SWOT-анализ" (рус) ',
  ' 14:40\xa0-\xa016:00 ',
  ' 25.01.2022 '],
 [' Менеджмент: концепции и технологии 21 века (рус) ',
  ' 16:20\xa0-\xa017:40 ',
  ' 25.01.2022 '],
 [' Дискретная математика (рус) ', ' 13:00\xa0-\xa014:20 ', ' 26.01.2022 '],
 [' Алгоритмы и структуры данных (рус) ',
  ' 14:40\xa0-\xa016:00 ',
  ' 26.01.2022 '],
 [' Английский язык для делового общения. Продвинутый курс - 2 (анг) ',
  ' 11:10\xa0-\xa012:30 ',
  ' 27.01.2022 '],
 [' Математический анализ (рус) ', ' 13:00\xa0-\xa014:20 ', ' 27.01.2022 '],
 [' Алгоритмы и структуры данных (рус) ',
  ' 11:10\xa0-\xa012:30 ',
  ' 28.01.2022 '],
 [' Алгоритмы и структуры данных (рус) ',
  ' 13:00\xa0-\xa014:20 ',
  ' 28.01.2022 '],
 [' Английский язык дл

In [158]:
# формируем время начала и конца пар в формате rc3339
for class_ in classes:
    class_[2] = class_[2][7:11]+'-'+class_[2][4:6]+'-'+class_[2][1:3]
    class_.append(class_[2]+'T'+class_[1][1:6]+':00+03:00')
    class_.append(class_[2]+'T'+class_[1][9:14]+':00+03:00')
classes

[[' Введение в программирование на Java (рус) ',
  ' 13:00\xa0-\xa014:20 ',
  '2022-01-24',
  '2022-01-24T13:00:00+03:00',
  '2022-01-24T14:20:00+03:00'],
 [' Дискретная математика (рус) ',
  ' 16:20\xa0-\xa017:40 ',
  '2022-01-24',
  '2022-01-24T16:20:00+03:00',
  '2022-01-24T17:40:00+03:00'],
 [' Математический анализ (рус) ',
  ' 11:10\xa0-\xa012:30 ',
  '2022-01-25',
  '2022-01-25T11:10:00+03:00',
  '2022-01-25T12:30:00+03:00'],
 [' Проектный семинар "SWOT-анализ" (рус) ',
  ' 14:40\xa0-\xa016:00 ',
  '2022-01-25',
  '2022-01-25T14:40:00+03:00',
  '2022-01-25T16:00:00+03:00'],
 [' Менеджмент: концепции и технологии 21 века (рус) ',
  ' 16:20\xa0-\xa017:40 ',
  '2022-01-25',
  '2022-01-25T16:20:00+03:00',
  '2022-01-25T17:40:00+03:00'],
 [' Дискретная математика (рус) ',
  ' 13:00\xa0-\xa014:20 ',
  '2022-01-26',
  '2022-01-26T13:00:00+03:00',
  '2022-01-26T14:20:00+03:00'],
 [' Алгоритмы и структуры данных (рус) ',
  ' 14:40\xa0-\xa016:00 ',
  '2022-01-26',
  '2022-01-26T14:40:00+0

In [161]:
# подгружаем всё через api(перед этим нужно войти в api и получить токен в первых нескольких фреймах)
for class_ in classes:
    class_event = {
        "summary":class_[0],
        "start":{
            "dateTime":class_[3],
            "timeZone":'Asia/Novosibirsk'
        },
        "end":{
            "dateTime":class_[4],
            "timeZone":'Asia/Novosibirsk'
        }
    }
    service.events().insert(calendarId='primary', body=class_event).execute()